# DSCI 100 Group Project Proposal

Team: Nicolas Mercuri, Tayler Whitehead, Chris Lukita, Stephanie Song

## Part 1: Introduction

Music plays a huge role in our experience of the world around us. Daily tasks like walking through campus, completing homework assignments, or simply cooking dinner after a long day, is a lot more enjoable while listening to your favourite song or playlist.

For our group project this semester, we decided to work with a dataset called [Spotify Tracks DB](https://tinyurl.com/3dnwcpe2). This dataset was downloaded from Kaggle.com and was parsed using Spotify's official API. The dataset has over 230,000 unique observations (tracks) and features 18 relevant variables, each describing a specific qualitative or quantitative characteristic of a track.

For our predictive question, we are hoping to determine if specific quantitative characteristics of a track (daceability, energy, duration (ms), and tempo) can help predict a new song's populatiry on Spotify.

## Part 2: Preliminary Exploratory Data Analysis

In [10]:
library(tidyverse)
library(repr)

spotify_raw_data = read_csv("data/SpotifyDataset.csv")

Parsed with column specification:
cols(
  genre = col_character(),
  artist_name = col_character(),
  track_name = col_character(),
  track_id = col_character(),
  popularity = col_double(),
  acousticness = col_double(),
  danceability = col_double(),
  duration_ms = col_double(),
  energy = col_double(),
  instrumentalness = col_double(),
  key = col_character(),
  liveness = col_double(),
  loudness = col_double(),
  mode = col_character(),
  speechiness = col_double(),
  tempo = col_double(),
  time_signature = col_character(),
  valence = col_double()
)

Warning message:
“1 parsing failure.
   row col   expected     actual                      file
115177  -- 18 columns 15 columns 'data/SpotifyDataset.csv'
”


## Part 3: Methods

**How will we coduct our data analysis?**

PLACEHOLDER

**Which variables are most important for our analysis?**

We expect to rely primarily on the following variables 
    qualitative columns
    - genre, artist_name, and track_name, as well as quantitative columns like danceability, energy, duration_ms, and tempo.

**How will we visualize results?**

As our prediction algorithm will be dealing with quantitative data, a scatterplot will be useful for visualizing the different datapoints within our dataset.

## Part 4: Expected Outcomes and Significance

**What do we expect to find?**

At first glance, we expect to find that new songs with higher ratings on both danceability and energy will likely score higher on popularity compared to songs that have lower ratings on these predictive variables. However, it will be interesting to dive further into this analysis and it our hope that our assumptions will be challenged and stretched.

**What impact could these findings have?**

Our findings could help artists and producers understand the metrics that make listeners really connect with a song. Making music is both an art and a science, and we hope that our analysis can provide artists and producers uncover insights to help them create new music that will resonate with listeners on Spotify and other streaming services.

**What future questions could this lead to?**

As this dataset is unique to the listening habits of Spotify users, it would be interesting to compare these results with data from other popular streaming services, like Apple Music, Amazon Music, and Youtube Music, respectively.